Limpeza dos Dados

In [1]:
import pandas as pd

# Carregar os dados
data = pd.read_csv('spotify_dataset.csv')

# Para colunas categóricas, preencha com a moda
for column in data.select_dtypes(include=['object']).columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

# Para colunas numéricas, você pode calcular o IQR
numeric_data = data.select_dtypes(exclude=['object'])

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

# Verifique outliers apenas para colunas numéricas
outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))) ## Revisar

# Crie um DataFrame apenas com os outliers
outliers_data = data[outliers.any(axis=1)]

# Remover outliers do DataFrame original
data_out = data[~outliers.any(axis=1)]

# Agora, `outliers_data` contém os outliers retirados
print(outliers_data)



                                  Nome da Faixa         Cantor ou Compositor  \
1                           Reborn in Blasphemy                    Dismember   
2                           Ultrasonic Meltdown  Curse of the Golden Vampire   
4                        The Masquerade is Over              George Shearing   
7                                      All Good                   De La Soul   
8                               Rats!Rats!Rats!                     Deftones   
..                                          ...                          ...   
698                         Comfort Zone; Pt. 1               Steven Halpern   
699  I Don't Stand a Ghost of a Chance With You                  Sonny Stitt   
700                     Let the Computer Decide                 Paul Gilbert   
701                          Brother Harold Dee               Porter Wagoner   
705                                        Read                      Product   

                                  Gener

In [2]:
data.describe()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000
mean,0.416577,0.528586,0.557864,0.180355,5.076487,0.198861,-10.548115,0.667139,0.086608,115.749734,3.886686,0.515737
std,0.369310,0.188623,0.282494,0.316745,3.478377,0.170141,5.464902,0.471571,0.089171,30.434357,0.443926,0.278639
min,0.000001,0.000000,0.003020,0.000000,0.000000,0.021000,-34.982000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.033325,0.402000,0.340250,0.000003,2.000000,0.096525,-13.254250,0.000000,0.036650,93.332750,4.000000,0.262000
50%,0.365000,0.549000,0.571500,0.000465,5.000000,0.125500,-9.869000,1.000000,0.049900,111.061500,4.000000,0.530000
75%,0.778750,0.671750,0.797500,0.174250,8.000000,0.258750,-6.350500,1.000000,0.096775,135.229750,4.000000,0.758750
max,0.996000,0.920000,0.998000,1.000000,11.000000,0.982000,-0.680000,1.000000,0.922000,209.139000,5.000000,0.980000


## Codificação de Categorias

In [3]:
# Primeiro, vamos identificar as colunas categóricas
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

# Remover as colunas "Nome da Faixa" e "Cantor ou Compositor" da lista
columns_to_exclude = ['Nome da Faixa', 'Cantor ou Compositor']
for col in columns_to_exclude:
    if col in categorical_columns:
        categorical_columns.remove(col)

# Verificar se as colunas foram removidas da lista
print("Colunas categóricas para codificação:", categorical_columns)

# Agora, vamos aplicar a codificação one-hot apenas às colunas selecionadas
data_encoded = pd.get_dummies(data, columns=categorical_columns)

print(data_encoded.head())  # exibir as primeiras linhas para ver a transformação
print("Total de colunas após a codificação:", len(data_encoded.columns))




Colunas categóricas para codificação: ['Genero', 'Quadrante']
                     Nome da Faixa         Cantor ou Compositor  acousticness  \
0  Bulldog Down in Sunny Tennessee                Charlie Poole      0.960000   
1              Reborn in Blasphemy                    Dismember      0.000008   
2              Ultrasonic Meltdown  Curse of the Golden Vampire      0.000195   
3                       Last Night                Little Walter      0.358000   
4           The Masquerade is Over              George Shearing      0.826000   

   danceability  energy  instrumentalness  key  liveness  loudness  mode  ...  \
0         0.649   0.390          0.000003    5     0.096   -12.237     1  ...   
1         0.246   0.985          0.839000    7     0.345    -4.002     1  ...   
2         0.275   0.990          0.654000    6     0.263    -2.375     1  ...   
3         0.458   0.443          0.000000    6     0.232   -10.422     0  ...   
4         0.522   0.132          0.848000    8

1. Expansão dos Valores Compostos:

In [4]:
# Criar um novo dataframe expandindo a coluna "Genero"
expanded_data = data.assign(Genero=data['Genero'].str.split(';')).explode('Genero')

# Remove espaços em branco adicionais (caso existam)
expanded_data['Genero'] = expanded_data['Genero'].str.strip()


2. Aplicar Codificação One-hot:

In [5]:
# Selecionar as colunas categóricas que você deseja codificar
cols_to_encode = ['Genero', 'Quadrante']

# Aplicar codificação one-hot
encoded_data = pd.get_dummies(expanded_data, columns=cols_to_encode, drop_first=False)

print(encoded_data.head())
print("Total de colunas após a codificação:", encoded_data.shape[1])


                     Nome da Faixa Cantor ou Compositor  acousticness  \
0  Bulldog Down in Sunny Tennessee        Charlie Poole      0.960000   
0  Bulldog Down in Sunny Tennessee        Charlie Poole      0.960000   
1              Reborn in Blasphemy            Dismember      0.000008   
1              Reborn in Blasphemy            Dismember      0.000008   
1              Reborn in Blasphemy            Dismember      0.000008   

   danceability  energy  instrumentalness  key  liveness  loudness  mode  ...  \
0         0.649   0.390          0.000003    5     0.096   -12.237     1  ...   
0         0.649   0.390          0.000003    5     0.096   -12.237     1  ...   
1         0.246   0.985          0.839000    7     0.345    -4.002     1  ...   
1         0.246   0.985          0.839000    7     0.345    -4.002     1  ...   
1         0.246   0.985          0.839000    7     0.345    -4.002     1  ...   

   Genero_R&B  Genero_Rap  Genero_Reggae  Genero_Religious  \
0       Fals

In [6]:
# Agrupando e somando os valores
df_agrupado = encoded_data.groupby(['Nome da Faixa', 'Cantor ou Compositor']).sum().reset_index()

# Convertendo valores maiores que 1 de volta para 1 (caso haja alguma soma que resultou em um valor > 1)
for col in df_agrupado.columns:
    if 'Genero_' in col or 'Quadrante_' in col:
        df_agrupado[col] = df_agrupado[col].apply(lambda x: 1 if x > 1 else x)


In [7]:
# Salvando o df_agrupado em CSV

df_agrupado.to_csv('spotify_dataset_grupado.csv', index=False)

### Normalização

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Criando o objeto scaler
scaler = MinMaxScaler()

# Lista de colunas numéricas para normalizar
# (Adicione ou remova colunas da lista conforme necessário)
colunas_numericas = [
    'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
    'loudness', 'mode', 'speechiness', 'tempo', 'valence'
]

# Aplicando a normalização
df_agrupado[colunas_numericas] = scaler.fit_transform(df_agrupado[colunas_numericas])


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version

Divisão de Dados:

In [10]:
# Convertendo de one-hot encoding para uma única coluna de classificação
df_agrupado['target'] = df_agrupado[['Quadrante_Q1', 'Quadrante_Q2', 'Quadrante_Q3', 'Quadrante_Q4']].idxmax(axis=1)


In [11]:
from sklearn.model_selection import train_test_split

# Primeira divisão: 70% para treinamento e 30% temporário
X_train, X_temp, y_train, y_temp = train_test_split(
    df_agrupado.drop('target', axis=1), 
    df_agrupado['target'], 
    test_size=0.3, 
    random_state=42, 
    stratify=df_agrupado['target']
)

# Divisão do conjunto temporário em validação e teste (metade de cada)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, 
    y_temp, 
    test_size=0.5, 
    random_state=42, 
    stratify=y_temp
)

f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inste

In [12]:
print("Proporção no Treinamento:\n", y_train.value_counts(normalize=True))
print("\nProporção na Validação:\n", y_val.value_counts(normalize=True))
print("\nProporção no Teste:\n", y_test.value_counts(normalize=True))

Proporção no Treinamento:
 target
Quadrante_Q4    0.261134
Quadrante_Q1    0.251012
Quadrante_Q2    0.248988
Quadrante_Q3    0.238866
Name: proportion, dtype: float64

Proporção na Validação:
 target
Quadrante_Q4    0.264151
Quadrante_Q2    0.254717
Quadrante_Q1    0.245283
Quadrante_Q3    0.235849
Name: proportion, dtype: float64

Proporção no Teste:
 target
Quadrante_Q4    0.264151
Quadrante_Q1    0.254717
Quadrante_Q2    0.245283
Quadrante_Q3    0.235849
Name: proportion, dtype: float64


In [15]:
df_agrupado_modelo  = df_agrupado
df_agrupado_modelo.to_csv('spotify_dataset_grupado_modelo.csv', index=False)

In [16]:
# Removendo as colunas categóricas e one-hot originais
X = df_agrupado.drop(columns=['Nome da Faixa', 'Cantor ou Compositor', 'Quadrante_Q1', 'Quadrante_Q2', 'Quadrante_Q3', 'Quadrante_Q4', 'target'])

# A coluna 'target' deve ser a última coluna do dataframe df_agrupado após a remoção das outras
y = df_agrupado['target']

# Se a coluna 'target' ainda não foi transformada numericamente, use LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)



f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [18]:
import numpy as np

# Verificar tipos de dados
print(X.dtypes)

# Verificar a presença de valores nulos
print(X.isnull().sum())

# Verificar a codificação da variável alvo
print(y)
print(np.unique(y))  # Deve mostrar um array de inteiros começando de 0

# Verificar o tamanho dos conjuntos de dados para garantir que eles estão alinhados
print(X.shape)
print(y.shape)


acousticness             float64
danceability             float64
energy                   float64
instrumentalness         float64
key                      float64
liveness                 float64
loudness                 float64
mode                     float64
speechiness              float64
tempo                    float64
time_signature             int64
valence                  float64
Genero_Avant-Garde         int64
Genero_Blues               int64
Genero_Children's          int64
Genero_Classical           int64
Genero_Comedy/Spoken       int64
Genero_Country             int64
Genero_Easy Listening      int64
Genero_Electronic          int64
Genero_Folk                int64
Genero_Holiday             int64
Genero_International       int64
Genero_Jazz                int64
Genero_Latin               int64
Genero_New Age             int64
Genero_Pop/Rock            int64
Genero_R&B                 int64
Genero_Rap                 int64
Genero_Reggae              int64
Genero_Rel

## Modelo XGBoost

In [20]:
import xgboost as xgb

# Crie o objeto do classificador XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=4, eval_metric='mlogloss')

# Treine o modelo
xgb_model.fit(X_train, y_train)

# Faça previsões no conjunto de validação
y_val_pred = xgb_model.predict(X_val)

# Avalie o modelo
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3], got ['Quadrante_Q1' 'Quadrante_Q2' 'Quadrante_Q3' 'Quadrante_Q4']

In [21]:
from sklearn.preprocessing import LabelEncoder

# Supondo que df_agrupado['target'] contém os rótulos de classe originais
le = LabelEncoder()
y_encoded = le.fit_transform(df_agrupado['target'])

# Verifique se as classes estão corretamente codificadas
print(np.unique(y_encoded))

[0 1 2 3]


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [ ]:
# Crie o objeto do classificador XGBoost, garantindo que num_class corresponda ao número de classes únicas
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_encoded)), eval_metric='mlogloss')

# Agora, treine o modelo com y_encoded
xgb_model.fit(X_train, y_encoded[train_index])  # Certifique-se de usar os índices corretos se você dividiu os dados


### REEXECUÇÃO PARTINDO DO DATASET AGRUPADO

In [23]:
# Supondo que 'df_agrupado' seja o seu DataFrame original.
# Primeiro, vamos remover as colunas que não são necessárias para o modelo
colunas_para_remover = ['Nome da Faixa', 'Cantor ou Compositor']  # Adicione outras colunas que não serão usadas
df_modelo = df_agrupado.drop(colunas_para_remover, axis=1)

# Se a coluna 'target' já estiver em formato numérico, ótimo. Se não, precisamos codificá-la.
# Vamos verificar se 'target' já é numérica:
print(df_modelo['target'].dtype)

# Se 'target' não for numérica (por exemplo, se o dtype for 'object'), use LabelEncoder para transformá-la.
if df_modelo['target'].dtype == 'object':
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_modelo['target'] = le.fit_transform(df_modelo['target'])


object


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [24]:
from sklearn.preprocessing import MinMaxScaler

# Cria o objeto scaler
scaler = MinMaxScaler()

# Lista das colunas numéricas para normalizar
colunas_numericas = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']

# Aplica a normalização
df_modelo[colunas_numericas] = scaler.fit_transform(df_modelo[colunas_numericas])

# Confirma que a normalização foi aplicada
print(df_modelo[colunas_numericas].describe())


       acousticness  danceability      energy  instrumentalness         key  \
count    706.000000    706.000000  706.000000      7.060000e+02  706.000000   
mean       0.177519      0.271518    0.204178      7.830877e-02    0.200252   
std        0.192592      0.161460    0.141199      1.605295e-01    0.179007   
min        0.000000      0.000000    0.000000      0.000000e+00    0.000000   
25%        0.010016      0.150802    0.105134      9.371585e-07    0.066667   
50%        0.115250      0.223761    0.176033      2.128962e-04    0.155556   
75%        0.315359      0.370408    0.257503      7.065574e-02    0.311111   
max        1.000000      1.000000    1.000000      1.000000e+00    1.000000   

         liveness    loudness        mode  speechiness       tempo  \
count  706.000000  706.000000  706.000000   706.000000  706.000000   
mean     0.102446    0.819338    0.237677     0.115395    0.255951   
std      0.120591    0.151081    0.214105     0.115837    0.136806   
min     

f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [25]:
from sklearn.model_selection import train_test_split

# Separa os recursos (X) e o target (y)
X = df_modelo.drop('target', axis=1)
y = df_modelo['target']

# Divide os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Confirma a divisão dos dados
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


(494, 37) (212, 37)
(494,) (212,)


In [26]:
import xgboost as xgb

# Cria o objeto do classificador XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y)), eval_metric='mlogloss')

# Treina o modelo
xgb_model.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred = xgb_model.predict(X_test)

# Avalia o modelo
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_boo

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       1.00      1.00      1.00        53
           2       1.00      1.00      1.00        50
           3       1.00      1.00      1.00        56

    accuracy                           1.00       212
   macro avg       1.00      1.00      1.00       212
weighted avg       1.00      1.00      1.00       212



f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\xgboost\data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_boo

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Crie o objeto do classificador Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treine o modelo
rf_model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred_rf = rf_model.predict(X_test)

# Avalie o modelo
print(accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       1.00      1.00      1.00        53
           2       1.00      1.00      1.00        50
           3       1.00      1.00      1.00        56

    accuracy                           1.00       212
   macro avg       1.00      1.00      1.00       212
weighted avg       1.00      1.00      1.00       212



f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version